In [257]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [258]:
col_name = ["ProblemId","C","C++","Go","Java","Javascript","Python","Python3"]
df = pd.read_csv('../../data/processed/success_score.csv')
df.rename(columns = {'Unnamed: 0':'problemId'}, inplace = True)
df

,problemId,C,C++,Go,Java,Javascript,Python,Python3
0,5b5c8cd7276e2200208fed62,0.708418,0.741621,0.674481,0.730613,0.674481,0.674481,0.763862
1,5b645f711b2cd40020b6c771,0.697211,0.780235,0.663106,0.698383,0.663106,0.890666,0.741500
2,5b649819c0b1b80020eb30e1,0.782912,0.729926,0.697609,0.702172,0.697609,0.697609,0.747527
3,5b64600d1b2cd40020b6c772,0.736975,0.736975,0.624211,0.701462,0.624211,0.624211,0.736975
4,5b6c3ec18cab100020e963b7,0.707936,0.753305,0.676529,0.738346,0.676529,0.676529,0.785069
...,...,...,...,...,...,...,...,...
199,63361b9a125f8a00222e06ef,0.660756,0.791391,0.660756,0.880797,0.660756,0.660756,0.660756
200,633620e3fe6e62002205e71e,0.669491,0.738728,0.669491,0.669491,0.669491,0.669491,0.669491
201,6336ad26ee62f10022c977c2,0.705785,0.747667,0.705785,0.705785,0.705785,0.705785,0.705785
202,6336b760ee62f10022c97c37,0.683354,0.759061,0.683354,0.683354,0.683354,0.683354,0.683354


In [259]:
df.describe()

,C,C++,Go,Java,Javascript,Python,Python3
count,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000
mean,0.704334,0.750738,0.688965,0.731316,0.684595,0.686746,0.740666
std,0.046229,0.057939,0.047726,0.067173,0.032995,0.041203,0.036715
min,0.622459,0.642960,0.622459,0.622459,0.622459,0.622459,0.622459
25%,0.677937,0.721152,0.660756,0.695437,0.660756,0.660756,0.723389
50%,0.702063,0.733347,0.681512,0.716736,0.679698,0.679698,0.731884
75%,0.724180,0.762232,0.705785,0.741552,0.705286,0.705785,0.747282
max,0.980708,0.999665,0.999997,0.994256,0.897982,0.988587,0.926574


In [260]:
# knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
X = df.drop('problemId', axis=1)
y = df['problemId']

In [261]:
def recommended(model,problemId):
    distances, indices = model.kneighbors(X_train)
    for i in range(len(distances)):
        if y_train.iloc[i] == problemId:
            return indices[i]

In [262]:
print(recommended(model, '6336b760ee62f10022c97c37'))

[202 198 174  90  45]


In [263]:
import pickle
def save_model(model, model_name):
    with open('../../models/'+model_name+'.pkl', 'wb') as file:
        pickle.dump(model, file)

In [264]:
save_model(model, 'knn_model')

In [265]:
#load model
with open('../../models/knn_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [266]:
print(recommended(model, '6336b760ee62f10022c97c37'))

[202 198 174  90  45]


In [267]:
new_model = model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
new_model.kneighbors_graph(X).toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [268]:
print(recommended(new_model, '6336b760ee62f10022c97c37'))

[202 198 174  90  45]


In [269]:
save_model(new_model, 'knn_model_new')

In [270]:
#load model
with open('../../models/knn_model_new.pkl', 'rb') as file:
    new_model = pickle.load(file)

print(recommended(new_model, '6336b760ee62f10022c97c37'))

[202 198 174  90  45]


In [271]:
from sklearn.neighbors import KNeighborsClassifier as KNN 
knn = KNN(n_neighbors = 3) 

In [272]:
knn.fit(X, y)

KNeighborsClassifier(n_neighbors=3)

In [273]:
to_predict = np.array([0.708418,0.741621,0.674481,0.730613,0.674481,0.674481,0.763862]).reshape(1, -1)

In [274]:
knn.predict(to_predict)

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array(['5b5c8cd7276e2200208fed62'], dtype=object)

In [275]:
def knn_recommend(problemId, n_neighbors,X,Y):
    to_predict = np.array(df[df['problemId']==problemId].drop('problemId', axis=1)).reshape(1, -1)
    distances, indices = knn.kneighbors(to_predict, n_neighbors=n_neighbors+1)
    rec = []
    for i in indices[0]:
        if Y.iloc[i] != problemId:
            rec.append(Y.iloc[i])
    return rec

In [276]:
knn_recommend('6336b760ee62f10022c97c37', 3,X,y)

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


['6335f3dd125f8a00222e06eb',
 '6256b3db9f86540024c5653f',
 '5f6ad8a34bf84b0022982aff']

In [277]:
df[df['problemId']=='5f6ad8a34bf84b0022982aff']

,problemId,C,C++,Go,Java,Javascript,Python,Python3
90,5f6ad8a34bf84b0022982aff,0.687162,0.761413,0.687162,0.687162,0.687162,0.687162,0.710996


In [278]:
df[df['problemId']=='6335f3dd125f8a00222e06eb']

,problemId,C,C++,Go,Java,Javascript,Python,Python3
198,6335f3dd125f8a00222e06eb,0.677158,0.74749,0.677158,0.677158,0.677158,0.677158,0.704655


In [279]:
df[df['problemId']=='6256b3db9f86540024c5653f']

,problemId,C,C++,Go,Java,Javascript,Python,Python3
174,6256b3db9f86540024c5653f,0.691353,0.771107,0.691353,0.702616,0.691353,0.691353,0.691353


In [280]:
v = []
for i in df['problemId']:
    v.append(knn_recommend(i, 3,X,y))

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does no

In [281]:
# check None in list
def checkNone(v):
    for i in v:
        if i == None:
            return True
    return False

checkNone(v)

False

In [282]:
from sklearn.neighbors import KNeighborsClassifier as KNN
def train_and_save(X, Y, n_neighbors, model_name):
    knn = KNN(n_neighbors = n_neighbors) 
    knn.fit(X, Y)
    save_model(knn, model_name)

In [283]:
train_and_save(X, y, 3, 'knn_model_3')

In [284]:
def load_model(model_name):
    with open('../../models/'+model_name+'.pkl', 'rb') as file:
        model = pickle.load(file)
    return model

In [285]:
def predict(model, problemId, n_neighbors):
    to_predict = np.array(df[df['problemId']==problemId].drop('problemId', axis=1)).reshape(1, -1)
    distances, indices = model.kneighbors(to_predict, n_neighbors=n_neighbors+1)
    rec = []
    for i in indices[0]:
        if y.iloc[i] != problemId:
            rec.append(y.iloc[i])
    return rec

In [286]:
predict(load_model('knn_model_3'), '6336b760ee62f10022c97c37', 3)

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


['6335f3dd125f8a00222e06eb',
 '6256b3db9f86540024c5653f',
 '5f6ad8a34bf84b0022982aff']

In [287]:
def recommended(model,problemId,X,Y):
    distances, indices = model.kneighbors(X)
    for i in range(len(distances)):
        if Y.iloc[i] == problemId:
            return indices[i]

In [288]:
from sklearn.neighbors import NearestNeighbors

In [292]:
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
nn.kneighbors_graph(X).toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [293]:
df.iloc[recommended(nn, '6336b760ee62f10022c97c37',X,y)]

,problemId,C,C++,Go,Java,Javascript,Python,Python3
202,6336b760ee62f10022c97c37,0.683354,0.759061,0.683354,0.683354,0.683354,0.683354,0.683354
198,6335f3dd125f8a00222e06eb,0.677158,0.747490,0.677158,0.677158,0.677158,0.677158,0.704655
174,6256b3db9f86540024c5653f,0.691353,0.771107,0.691353,0.702616,0.691353,0.691353,0.691353
90,5f6ad8a34bf84b0022982aff,0.687162,0.761413,0.687162,0.687162,0.687162,0.687162,0.710996
45,5ca063dd08f90d00223749ea,0.691618,0.779026,0.691618,0.691618,0.691618,0.691618,0.705099


In [294]:
from sklearn.neighbors import KNeighborsClassifier as KNN
def train_knn(X,Y, n_neighbors, algorithm, model_name):
    knn = KNN(n_neighbors = n_neighbors+1, algorithm = algorithm) 
    knn.fit(X, Y)
    save_model(knn, model_name)
    

In [295]:
train_knn(X, y, 3, 'ball_tree', 'knn_model_3_ball_tree')

In [296]:
import numpy as np
def knn_recommend(problemId, n_neighbors,df, knn):
    X = df.drop('problemId', axis=1)
    Y = df['problemId']
    to_predict = np.array(df[df['problemId']==problemId].drop('problemId', axis=1)).reshape(1, -1)
    distances, indices = knn.kneighbors(to_predict, n_neighbors=n_neighbors+1)
    rec = []
    for i in indices[0]:
        if Y.iloc[i] != problemId:
            rec.append(Y.iloc[i])
    return rec

In [297]:
knn_recommend('6336b760ee62f10022c97c37', 3,df, load_model('knn_model_3_ball_tree'))

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


['6335f3dd125f8a00222e06eb',
 '6256b3db9f86540024c5653f',
 '5f6ad8a34bf84b0022982aff']

In [299]:
t = []
for i in df['problemId']:
    t.append(knn_recommend(i, 3,df, load_model('knn_model_3_ball_tree')))

c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\Recommendation-system\env\lib\site-packages\sklearn\base.py:409: UserWarning: X does no

In [300]:
checkNone(t)

False